# Generate APIspec 

In [3]:
!pip install APIspec

In [4]:
!pip install Flask

In [5]:
!pip install marshmallow

In [7]:
!pip install apispec-webframeworks

In [9]:
!pip install -U apispec[marshmallow]

In [13]:
import uuid
from apispec import APISpec
from apispec.ext.marshmallow import MarshmallowPlugin
from apispec_webframeworks.flask import FlaskPlugin
from flask import Flask
from marshmallow import Schema, fields


# Create an APISpec
spec = APISpec(
    title="Swagger Petstore",
    version="1.0.0",
    openapi_version="3.0.2",
    plugins=[FlaskPlugin(), MarshmallowPlugin()],
)

# Optional marshmallow support
class CategorySchema(Schema):
    id = fields.Int()
    name = fields.Str(required=True)


class PetSchema(Schema):
    categories = fields.List(fields.Nested(CategorySchema))
    name = fields.Str()


# Optional security scheme support
api_key_scheme = {"type": "apiKey", "in": "header", "name": "X-API-Key"}
spec.components.security_scheme("ApiKeyAuth", api_key_scheme)


# Optional Flask support
app = Flask(__name__)


@app.route("/random")
def random_pet():
    """A cute furry animal endpoint.
    ---
    get:
      description: Get a random pet
      security:
        - ApiKeyAuth: []
      responses:
        200:
          description: Return a pet
          content:
            application/json:
              schema: PetSchema
    """
    # Hardcoded example data
    pet_data = {
        "name": "sample_pet_" + str(uuid.uuid1()),
        "categories": [{"id": 1, "name": "sample_category"}],
    }
    return PetSchema().dump(pet_data)


# Register the path and the entities within it
with app.test_request_context():
    spec.path(view=random_pet)


In [14]:
import json

print(json.dumps(spec.to_dict(), indent=2))

{
  "paths": {
    "/random": {
      "get": {
        "description": "Get a random pet",
        "security": [
          {
            "ApiKeyAuth": []
          }
        ],
        "responses": {
          "200": {
            "description": "Return a pet",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          }
        }
      }
    }
  },
  "info": {
    "title": "Swagger Petstore",
    "version": "1.0.0"
  },
  "openapi": "3.0.2",
  "components": {
    "schemas": {
      "Category": {
        "type": "object",
        "properties": {
          "id": {
            "type": "integer"
          },
          "name": {
            "type": "string"
          }
        },
        "required": [
          "name"
        ]
      },
      "Pet": {
        "type": "object",
        "properties": {
          "categories": {
            "type": "array

In [15]:
print(spec.to_yaml())

components:
  schemas:
    Category:
      properties:
        id:
          type: integer
        name:
          type: string
      required:
      - name
      type: object
    Pet:
      properties:
        categories:
          items:
            $ref: '#/components/schemas/Category'
          type: array
        name:
          type: string
      type: object
  securitySchemes:
    ApiKeyAuth:
      in: header
      name: X-API-Key
      type: apiKey
info:
  title: Swagger Petstore
  version: 1.0.0
openapi: 3.0.2
paths:
  /random:
    get:
      description: Get a random pet
      responses:
        '200':
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Pet'
          description: Return a pet
      security:
      - ApiKeyAuth: []

